In [1]:
from transformers import ConvNextForImageClassification

model = ConvNextForImageClassification.from_pretrained(
    "ahsanjavid/convnext-tiny-finetuned-cifar10"
)

config.json:   0%|          | 0.00/977 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/111M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/111M [00:00<?, ?B/s]

In [6]:
from transformers import ConvNextForImageClassification
import torch
import torch.nn as nn
def load_hf_convnext_backbone(hf_repo="ahsanjavid/convnext-tiny-finetuned-cifar10", to_gray=True):
    model = ConvNextForImageClassification.from_pretrained(hf_repo)
    backbone = model.convnext  # Extract the backbone (without the classifier head)

    if to_gray:
        # Convert first conv layer from (3,...) → (1,...)
        old_conv = backbone.embeddings.patch_embeddings
        new_conv = nn.Conv2d(
            in_channels=1,
            out_channels=old_conv.out_channels,
            kernel_size=old_conv.kernel_size,
            stride=old_conv.stride,
            padding=old_conv.padding,
            bias=old_conv.bias is not None,
        )

        # Average RGB weights to grayscale
        with torch.no_grad():
            new_conv.weight[:] = old_conv.weight.mean(dim=1, keepdim=True)
            if old_conv.bias is not None:
                new_conv.bias[:] = old_conv.bias
        backbone.embeddings.patch_embeddings = new_conv

    return backbone


In [7]:
load_hf_convnext_backbone()

ConvNextModel(
  (embeddings): ConvNextEmbeddings(
    (patch_embeddings): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
    (layernorm): ConvNextLayerNorm()
  )
  (encoder): ConvNextEncoder(
    (stages): ModuleList(
      (0): ConvNextStage(
        (downsampling_layer): Identity()
        (layers): Sequential(
          (0): ConvNextLayer(
            (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (layernorm): ConvNextLayerNorm()
            (pwconv1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELUActivation()
            (pwconv2): Linear(in_features=384, out_features=96, bias=True)
            (drop_path): Identity()
          )
          (1): ConvNextLayer(
            (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (layernorm): ConvNextLayerNorm()
            (pwconv1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELUAc

In [1]:
import timm

# List all ConvNeXt model names
all_models = timm.list_models('convnext*')
print("🔍 All ConvNeXt models:")
for name in all_models:
    print("  -", name)

# List only pretrained ConvNeXt models
pretrained_models = timm.list_models('convnext*', pretrained=True)
print("\n✅ Pretrained ConvNeXt models:")
for name in pretrained_models:
    print("  -", name)

🔍 All ConvNeXt models:
  - convnext_atto
  - convnext_atto_ols
  - convnext_atto_rms
  - convnext_base
  - convnext_femto
  - convnext_femto_ols
  - convnext_large
  - convnext_large_mlp
  - convnext_nano
  - convnext_nano_ols
  - convnext_pico
  - convnext_pico_ols
  - convnext_small
  - convnext_tiny
  - convnext_tiny_hnf
  - convnext_xlarge
  - convnext_xxlarge
  - convnext_zepto_rms
  - convnext_zepto_rms_ols
  - convnextv2_atto
  - convnextv2_base
  - convnextv2_femto
  - convnextv2_huge
  - convnextv2_large
  - convnextv2_nano
  - convnextv2_pico
  - convnextv2_small
  - convnextv2_tiny

✅ Pretrained ConvNeXt models:
  - convnext_atto.d2_in1k
  - convnext_atto_ols.a2_in1k
  - convnext_base.clip_laion2b
  - convnext_base.clip_laion2b_augreg
  - convnext_base.clip_laion2b_augreg_ft_in1k
  - convnext_base.clip_laion2b_augreg_ft_in12k
  - convnext_base.clip_laion2b_augreg_ft_in12k_in1k
  - convnext_base.clip_laion2b_augreg_ft_in12k_in1k_384
  - convnext_base.clip_laiona
  - convnext_

In [4]:
# backbone="convnext_fb_in22k_ft_in1k"
# backbone="convnext_fb_in1k"
backbone="convnext_gaussian"
if backbone.startswith('convnext'):
    tokens = backbone.split('_')
    init = tokens[-1] if len(tokens) > 1 else 'fb_in1k'

    if init == 'fb_in22k_ft_in1k':
        model_name = 'convnext_tiny.fb_in22k_ft_in1k'
        pretrained = True
    elif init == 'fb_in1k':
        model_name = 'convnext_tiny.fb_in1k'
        pretrained = True
    else:
        model_name = 'convnext_tiny'
        pretrained = False
print(f"Using model: {model_name}, pretrained: {pretrained}")

Using model: convnext_tiny, pretrained: False
